# Hyperspectral Dataset for Autoencoder

All imports

In [1]:
from scipy.io import loadmat
from h5py import File as h5file
from os import listdir
from os.path import join
from numpy.random import permutation
from tqdm import tqdm_notebook as tqdm
from math import ceil
import numpy as np

/home/entropy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Configuration parameters

In [2]:
source_dir = './data/'
destination_name = './HyperspectralData.hdf5'
image_shape = (254, 510, 15)
blocksize = 1024*1024*1024 # 1GB * 15 * 2 ~ 30 GB

Find all input files.

In [3]:
filenames = [join(source_dir, name) for name in listdir(source_dir) if name.endswith('.mat')]

Create the destination dataset.

In [4]:
total_pixel_count = len(filenames) * image_shape[0] * image_shape[1]
destination_file = h5file(destination_name, mode='x', driver=None)
dataset = destination_file.create_dataset(name='data',
                                              shape=(total_pixel_count, image_shape[2]),
                                              dtype='f',
                                              data=None,
                                              compression='gzip',
                                              compression_opts=6)

Fill the dataset.

In [5]:
image_pixel_count = image_shape[0] * image_shape[1]
for i, filename in enumerate(tqdm(permutation(filenames))):
    data = loadmat(filename)['data'].reshape(image_pixel_count, image_shape[2])
    dataset[i*image_pixel_count:(i+1)*image_pixel_count] = permutation(data)

HBox(children=(IntProgress(value=0, max=19), HTML(value='')))

Save the dataset.

In [6]:
dataset.flush()

Shuffel the data.

In [7]:
num_iterations = ceil(total_pixel_count/blocksize)

for i in tqdm(range(num_iterations)):
    dataset[i*blocksize:(i+1)*blocksize] = permutation(np.array(dataset[i*blocksize:(i+1)*blocksize]))

HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

Save all data and close files.

In [ ]:
dataset.flush()
destination_file.close()